In [1]:
import sys
import os
import time
import glob
import numpy as np
import matplotlib.pyplot as plt
import mne
from mne.time_frequency import tfr_multitaper
from matplotlib.patches import Rectangle
import scipy.stats as stats
import logging
import time


In [2]:
dir = os.getcwd()
print(dir)

c:\Users\Ania\Desktop\studia\courses\6sem\thesis\RM


In [3]:
#Loading all channels and subsets

ch_rm = ['Fp1', 'Fpz', 'Fp2', 'AFz', 'AF3', 'AF7', 'AF4', 'AF8', 'F7', 'F5', 'F3', 'F1', 'Fz', 'F2', 
         'F4', 'F6', 'F8', 'FT7', 'FC5', 'FC3', 'FC1', 'FCz', 'FC2', 'FC4', 'FC6', 'FT8', 'O1', 'C5', 
         'C3', 'C1', 'Cz', 'C2', 'C4', 'C6', 'O2', 'CP5', 'CP3', 'CP1', 'CPz', 'CP2', 'CP4', 'CP6', 'P7', 
         'P5', 'P3', 'P1', 'Pz', 'P2', 'P4', 'P6', 'P8', 'PO7', 'PO3', 'POz', 'PO4', 'PO8']

f_rm = ['Fp1', 'Fpz', 'Fp2', 'F7', 'F5', 'F3', 'F1', 'Fz', 'F2', 'F4', 'F6', 'F8', 'FT7', 'FC5', 'FC3', 'FC1', 'FCz', 'FC2', 'FC4', 'FC6', 'FT8']

c_rm = ["C5", 'C3', 'C1', 'Cz', 'C2', 'C4', 'C6', 'CP5', 'CP3', 'CP1', 'CPz', 'CP2', 'CP4', 'CP6']

p_rm = ["P7",  'P5', 'P3', 'P1', 'Pz', 'P2', 'P4', 'P6', 'P8', 'PO7', 'PO3', 'POz', 'PO4', 'PO8']

la_rm = ["F1", "F3", "F5", "FC1", "FC3", "FC5"]

ra_rm = ["F2", "F4","F6", "FC2", "FC4", "FC6"]

lp_rm = ["CP1", "CP3", "CP5", "P1", "P3", "P5"]

rp_rm = ["CP2", "CP4", "CP6", "P2", "P4", "P6"]

In [4]:
def mne_multitaper_power(epochs, freqs, n_cycles, tmin, tmax):
    
    #Calculates power using multitaper method.
    power = epochs.compute_tfr(
        method="multitaper",
        freqs=freqs,
        n_cycles=n_cycles,
        decim=10,
        return_itc=False,
        average=True,
        picks="eeg",
        n_jobs=-1,
    )
    return power.crop(tmin=tmin, tmax=tmax, include_tmax=True)

In [5]:
def categorize_stimulus(stimulus):
    """Categorizes a stimulus into supersets based on its code."""
    supersets = []

    if stimulus in ["121", "123", "125", "127", "101", "103", "105", "107"]:  # Irregular Rhythm
        supersets.append("XR")
    if stimulus in ["122", "124", "126", "128", "102", "104", "106", "108"]:  # Regular Rhythm
        supersets.append("XR")

    if stimulus in ["121", "122", "123", "124", "101", "102", "103", "104", "105", "106", "107", "108"]:  # Non-Ambiguous
        supersets.append("XA")
    if stimulus in ["125", "126", "127", "128"]:  # Ambiguous for second epoch
        supersets.append("XA") 
        
    if stimulus in ["121", "122", "125", "126", "101", "102", "105", "106"]:  # Subject-first
        supersets.append("XF")
    if stimulus in ["123", "124", "127", "128", "103", "104", "107", "108"]:  # Object-first
        supersets.append("XF")

    return supersets

In [6]:
def mne_multitaper_concate(iStims, path, freqs, fmin, fmax, tmin, tmax):
    # Initialize logging
    logging.basicConfig(filename="debug.log", level=logging.DEBUG)
    logging.info(f"Processing stimuli: {iStims}")

    tic = time.perf_counter()
    logging.info(f"Start time: {tic}")

    n_cycles = freqs / 2.
    all_epochs = []
    file_list = []
    all_annotations = None

    # Initialize ambiguity tracking variables (remove global variables)
    XR = set()
    XA = set()
    XF = set()

    # Iterate over selected stimuli and participant numbers
    for iStim in iStims:
        for i in [1, 2, 6, 8, 9, 10, 11, 12, 13, 14, 16, 18, 21, 23, 28, 30, 32]:
            dir_path = os.path.join(path, f"RM{str(i).zfill(2)}/")
        
            if os.path.exists(dir_path):
                # List files in the directory and filter by stimulus
                file_list.extend([dir_path + f for f in os.listdir(dir_path) if iStim in f])
 
        supersets = categorize_stimulus(iStim)
        if "XR" in supersets:
            XR.add(iStim)
        if "XA" in supersets:
            XA.add(iStim)
        if "XF" in supersets:
            XF.add(iStim)
    
    for file in file_list:
        file_epoch = mne.read_epochs(file, verbose=False)
        file_epoch.apply_baseline((-0.5, 0), verbose=False)
        
        if file_epoch.annotations is not None:
            if all_annotations is None:
                all_annotations = file_epoch.annotations
            else:
                all_annotations += file_epoch.annotations
        
        all_epochs.append(file_epoch)
    
    # Concatenating epoch files
    concatenated_epochs = mne.concatenate_epochs(all_epochs)

    if all_annotations is not None:
        concatenated_epochs.set_annotations(all_annotations)

    # Power analysis
    power = mne_multitaper_power(concatenated_epochs, freqs, freqs / 2., tmin, tmax)

    cond = "unknown"  # default value
    rhythm = "unknown" 
    # Assign condition labels based on stimuli
    if path == "RM_1/RM_1st":    
        if iStims == ["101", "103","105","107"]:
            cond = "first_irr"
        elif iStims == ["102", "104","106","108"]:
            cond = "first_reg"
    if path == "RM_2/":         
        if iStims == ["121","122","125","126"]:
            cond = "sec_subj"
        elif iStims == ["123","124","127","128"]:
            cond = "sec_obj"
        elif iStims == ["121","123","125","127"]:
            cond = "sec_irr"
        elif iStims == ["122","124","126","128"]:
            cond = "sec_reg"
        else:
            cond = "mixed"

    # Determining ambiguity category
    ambiguity = "non_ambiguous"  # Default value
    if any(categorize_stimulus(stim) for stim in iStims): #using the categorization function to check ambiguity
        ambiguity = "ambiguous"

    end_time = time.perf_counter()
    logging.info(f"End time: {end_time}")
    logging.info(f"Total runtime: {end_time - tic:.2f} seconds")


    final_array = power.crop(tmin=tmin, tmax=tmax, include_tmax=True).data  # shape: chan x freq x time

    tfr_name = f"RM_{ambiguity}{cond}{rhythm}-tfr.h5"
    npy_name = f"RM_{ambiguity}{cond}{rhythm}.npy"

    os.chdir("Concatenated_events/Grand_Average_Excluded/")
    mne.time_frequency.write_tfrs(tfr_name, power, overwrite=True, verbose=True)
    np.save(npy_name, final_array)

    print(f"Saved: {tfr_name} and {npy_name}")

    toc = time.perf_counter()
    print(f"Processed the file in {toc - tic:0.4f} seconds")

    return cond, ambiguity, power

In [ ]:
import os
import glob
import mne
import numpy as np
import pandas as pd

file_paths = glob.glob(os.path.join(dir, "**", "*.fif"), recursive=True)

data_groups = {
    "f_rm": ['Fp1', 'Fpz', 'Fp2', 'F7', 'F5', 'F3', 'F1', 'Fz', 'F2', 'F4', 'F6', 'F8', 'FT7', 'FC5', 'FC3', 'FC1', 'FCz', 'FC2', 'FC4', 'FC6', 'FT8'],
    "c_rm": ["C5", 'C3', 'C1', 'Cz', 'C2', 'C4', 'C6', 'CP5', 'CP3', 'CP1', 'CPz', 'CP2', 'CP4', 'CP6'],
    "p_rm": ["P7",  'P5', 'P3', 'P1', 'Pz', 'P2', 'P4', 'P6', 'P8', 'PO7', 'PO3', 'POz', 'PO4', 'PO8'],
    "la_rm": ["F1", "F3", "F5", "FC1", "FC3", "FC5"],
    "ra_rm": ["F2", "F4","F6", "FC2", "FC4", "FC6"],
    "lp_rm": ["CP1", "CP3", "CP5", "P1", "P3", "P5"],
    "rp_rm": ["CP2", "CP4", "CP6", "P2", "P4", "P6"]
}

merged_data = {group: {} for group in data_groups}

for file_path in file_paths:
    try:
        raw = mne.read_epochs(file_path, preload=True, verbose=False)
        data_type = "Epochs"
    except Exception as e:
        print(f"Skipping unsupported file format: {file_path} ({e})")
        continue

    print(f"Processing {file_path} as {data_type}")

    events = raw.events  
    event_ids = raw.event_id

    if len(events) == 0:
        print(f"No events found in {file_path}")
        continue

    event_df = pd.DataFrame(events, columns=["sample", "previous", "event_id"])
    event_df["participant"] = os.path.basename(file_path)

    for group, electrodes in data_groups.items():
        for electrode in electrodes:
            if electrode not in raw.ch_names:
                continue

            # Ensure dictionary entry exists before appending
            if electrode not in merged_data[group]:
                merged_data[group][electrode] = []

            idx = raw.ch_names.index(electrode)
        
            electrode_data = raw.get_data(picks=[idx]).mean(axis=0)  # Average over epochs
        
            event_df[electrode] = electrode_data.flatten()
            merged_data[group][electrode].append(event_df)
    
# Concatenate all data per electrode group
final_data = {}
for group in merged_data:
    final_data[group] = {elec: pd.concat(merged_data[group][elec]) for elec in merged_data[group] if merged_data[group][elec]}

output_dir = "Concatenated_events/Concatenated_per_electrode/"
os.makedirs(output_dir, exist_ok=True)

for group, electrodes in final_data.items():
    group_dir = os.path.join(output_dir, group)
    os.makedirs(group_dir, exist_ok=True)

    for elec, df in electrodes.items():
        df.to_csv(os.path.join(group_dir, f"merged_{elec}.csv"), index=False)

print("Merging complete.")

Processing c:\Users\Ania\Desktop\studia\courses\6sem\thesis\RM\RM_1\RM_1st\RM01\RM01_101_epo.fif as Epochs


ValueError: Length of values (1646) does not match length of index (25)

In [ ]:
import os
import glob
import mne
import numpy as np
import pandas as pd

file_paths = glob.glob(os.path.join(dir, "**", "*.fif"), recursive=True)
 

data_groups = {
    "f_rm": ['Fp1', 'Fpz', 'Fp2', 'F7', 'F5', 'F3', 'F1', 'Fz', 'F2', 'F4', 'F6', 'F8', 'FT7', 'FC5', 'FC3', 'FC1', 'FCz', 'FC2', 'FC4', 'FC6', 'FT8'],
    "c_rm": ["C5", 'C3', 'C1', 'Cz', 'C2', 'C4', 'C6', 'CP5', 'CP3', 'CP1', 'CPz', 'CP2', 'CP4', 'CP6'],
    "p_rm": ["P7",  'P5', 'P3', 'P1', 'Pz', 'P2', 'P4', 'P6', 'P8', 'PO7', 'PO3', 'POz', 'PO4', 'PO8'],
    "la_rm": ["F1", "F3", "F5", "FC1", "FC3", "FC5"],
    "ra_rm": ["F2", "F4","F6", "FC2", "FC4", "FC6"],
    "lp_rm": ["CP1", "CP3", "CP5", "P1", "P3", "P5"],
    "rp_rm": ["CP2", "CP4", "CP6", "P2", "P4", "P6"]
}


for file_path in file_paths:
    try:
        raw = mne.io.read_raw_fif(file_path, preload=True, verbose=False)
        data_type = "Raw"
    except ValueError:
        try:
            raw = mne.read_epochs(file_path, preload=True, verbose=False)
            data_type = "Epochs"
        except ValueError:
            try:
                raw = mne.read_evokeds(file_path, verbose=False)
                data_type = "Evoked"
            except ValueError:
                print(f"Skipping unsupported file format: {file_path}")
                continue  # Skip unsupported files

    print(f"Processing {file_path} as {data_type}")

    try:
        events = mne.find_events(raw, stim_channel=None, output="step", shortest_event=1)
        event_ids = {str(event): event for event in np.unique(events[:, -1])}
    except Exception as e:
        print(f"Could not extract events from {file_path}: {e}")
        continue 

    event_df = pd.DataFrame(events, columns=["sample", "previous", "event_id"])
    event_df["participant"] = os.path.basename(file_path)

    for group, electrodes in data_groups.items():
        for electrode in electrodes:
            if electrode not in raw.ch_names:
                continue

            if electrode not in merged_data[group]:
                merged_data[group][electrode] = []

            idx = raw.ch_names.index(electrode)
            
            
            if data_type == "Epochs":
                electrode_data = raw.get_data(picks=[idx]).mean(axis=0)  # Average over epochs
            else:
                electrode_data = raw.get_data(picks=[idx])

            event_df[electrode] = electrode_data.flatten()
            merged_data[group][electrode].append(event_df)

# Concatenate all data per electrode group
final_data = {}
for group in merged_data:
    final_data[group] = {elec: pd.concat(merged_data[group][elec]) for elec in merged_data[group] if merged_data[group][elec]}


output_dir = "Concatenated_events/Concatenated_per_electrode/"
os.makedirs(output_dir, exist_ok=True)

for group, electrodes in final_data.items():
    group_dir = os.path.join(output_dir, group)
    os.makedirs(group_dir, exist_ok=True)

    for elec, df in electrodes.items():
        df.to_csv(os.path.join(group_dir, f"merged_{elec}.csv"), index=False)

print("Merging complete.")

C:\Users\Ania\AppData\Local\Temp\ipykernel_21216\702253907.py:23: RuntimeWarning: This filename (c:\Users\Ania\Desktop\studia\courses\6sem\thesis\RM\RM_1\RM_1st\RM01\RM01_101_epo.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif(file_path, preload=True, verbose=False)
C:\Users\Ania\AppData\Local\Temp\ipykernel_21216\702253907.py:23: RuntimeWarning: This filename (c:\Users\Ania\Desktop\studia\courses\6sem\thesis\RM\RM_1\RM_1st\RM01\RM01_102_epo.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif(file_path, preload=True, verbose=False)
C:\Users\Ania\AppData\Local\Temp\ipykernel

Processing c:\Users\Ania\Desktop\studia\courses\6sem\thesis\RM\RM_1\RM_1st\RM01\RM01_101_epo.fif as Epochs
Could not extract events from c:\Users\Ania\Desktop\studia\courses\6sem\thesis\RM\RM_1\RM_1st\RM01\RM01_101_epo.fif: No stim channels found, but the raw object has annotations. Consider using mne.events_from_annotations to convert these to events.
Processing c:\Users\Ania\Desktop\studia\courses\6sem\thesis\RM\RM_1\RM_1st\RM01\RM01_102_epo.fif as Epochs
Could not extract events from c:\Users\Ania\Desktop\studia\courses\6sem\thesis\RM\RM_1\RM_1st\RM01\RM01_102_epo.fif: No stim channels found, but the raw object has annotations. Consider using mne.events_from_annotations to convert these to events.
Processing c:\Users\Ania\Desktop\studia\courses\6sem\thesis\RM\RM_1\RM_1st\RM01\RM01_103_epo.fif as Epochs
Could not extract events from c:\Users\Ania\Desktop\studia\courses\6sem\thesis\RM\RM_1\RM_1st\RM01\RM01_103_epo.fif: No stim channels found, but the raw object has annotations. Consid

C:\Users\Ania\AppData\Local\Temp\ipykernel_21216\702253907.py:23: RuntimeWarning: This filename (c:\Users\Ania\Desktop\studia\courses\6sem\thesis\RM\RM_1\RM_1st\RM01\RM01_105_epo.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif(file_path, preload=True, verbose=False)
C:\Users\Ania\AppData\Local\Temp\ipykernel_21216\702253907.py:23: RuntimeWarning: This filename (c:\Users\Ania\Desktop\studia\courses\6sem\thesis\RM\RM_1\RM_1st\RM01\RM01_106_epo.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif(file_path, preload=True, verbose=False)
C:\Users\Ania\AppData\Local\Temp\ipykernel

Processing c:\Users\Ania\Desktop\studia\courses\6sem\thesis\RM\RM_1\RM_1st\RM01\RM01_107_epo.fif as Epochs
Could not extract events from c:\Users\Ania\Desktop\studia\courses\6sem\thesis\RM\RM_1\RM_1st\RM01\RM01_107_epo.fif: No stim channels found, but the raw object has annotations. Consider using mne.events_from_annotations to convert these to events.
Processing c:\Users\Ania\Desktop\studia\courses\6sem\thesis\RM\RM_1\RM_1st\RM01\RM01_108_epo.fif as Epochs
Could not extract events from c:\Users\Ania\Desktop\studia\courses\6sem\thesis\RM\RM_1\RM_1st\RM01\RM01_108_epo.fif: No stim channels found, but the raw object has annotations. Consider using mne.events_from_annotations to convert these to events.
Processing c:\Users\Ania\Desktop\studia\courses\6sem\thesis\RM\RM_1\RM_1st\RM01\RM01_201_epo.fif as Epochs
Could not extract events from c:\Users\Ania\Desktop\studia\courses\6sem\thesis\RM\RM_1\RM_1st\RM01\RM01_201_epo.fif: No stim channels found, but the raw object has annotations. Consid

C:\Users\Ania\AppData\Local\Temp\ipykernel_21216\702253907.py:23: RuntimeWarning: This filename (c:\Users\Ania\Desktop\studia\courses\6sem\thesis\RM\RM_1\RM_1st\RM01\RM01_108_epo.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif(file_path, preload=True, verbose=False)
C:\Users\Ania\AppData\Local\Temp\ipykernel_21216\702253907.py:23: RuntimeWarning: This filename (c:\Users\Ania\Desktop\studia\courses\6sem\thesis\RM\RM_1\RM_1st\RM01\RM01_201_epo.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif(file_path, preload=True, verbose=False)
C:\Users\Ania\AppData\Local\Temp\ipykernel

...

Processing c:\Users\Ania\Desktop\studia\courses\6sem\thesis\RM\RM_1\RM_1st\RM09\RM09_108_epo.fif as Epochs
Could not extract events from c:\Users\Ania\Desktop\studia\courses\6sem\thesis\RM\RM_1\RM_1st\RM09\RM09_108_epo.fif: No stim channels found, but the raw object has annotations. Consider using mne.events_from_annotations to convert these to events.
Processing c:\Users\Ania\Desktop\studia\courses\6sem\thesis\RM\RM_1\RM_1st\RM09\RM09_201_epo.fif as Epochs
Could not extract events from c:\Users\Ania\Desktop\studia\courses\6sem\thesis\RM\RM_1\RM_1st\RM09\RM09_201_epo.fif: No stim channels found, but the raw object has annotations. Consider using mne.events_from_annotations to convert these to events.
Processing c:\Users\Ania\Desktop\studia\courses\6sem\thesis\RM\RM_1\RM_1st\RM09\RM09_202_epo.fif as Epochs
Could not extract events from c:\Users\Ania\Desktop\studia\courses\6sem\thesis\RM\RM_1\RM_1st\RM09\RM09_202_epo.fif: No stim channels found, but the raw object has annotations. Consid

C:\Users\Ania\AppData\Local\Temp\ipykernel_21216\702253907.py:23: RuntimeWarning: This filename (c:\Users\Ania\Desktop\studia\courses\6sem\thesis\RM\RM_1\RM_1st\RM09\RM09_201_epo.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif(file_path, preload=True, verbose=False)
C:\Users\Ania\AppData\Local\Temp\ipykernel_21216\702253907.py:23: RuntimeWarning: This filename (c:\Users\Ania\Desktop\studia\courses\6sem\thesis\RM\RM_1\RM_1st\RM09\RM09_202_epo.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif(file_path, preload=True, verbose=False)
C:\Users\Ania\AppData\Local\Temp\ipykernel

Processing c:\Users\Ania\Desktop\studia\courses\6sem\thesis\RM\RM_1\RM_1st\RM09\RM09_205_epo.fif as Epochs
Could not extract events from c:\Users\Ania\Desktop\studia\courses\6sem\thesis\RM\RM_1\RM_1st\RM09\RM09_205_epo.fif: No stim channels found, but the raw object has annotations. Consider using mne.events_from_annotations to convert these to events.
Processing c:\Users\Ania\Desktop\studia\courses\6sem\thesis\RM\RM_1\RM_1st\RM10\RM10_101_epo.fif as Epochs
Could not extract events from c:\Users\Ania\Desktop\studia\courses\6sem\thesis\RM\RM_1\RM_1st\RM10\RM10_101_epo.fif: No stim channels found, but the raw object has annotations. Consider using mne.events_from_annotations to convert these to events.


C:\Users\Ania\AppData\Local\Temp\ipykernel_21216\702253907.py:23: RuntimeWarning: This filename (c:\Users\Ania\Desktop\studia\courses\6sem\thesis\RM\RM_1\RM_1st\RM10\RM10_101_epo.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif(file_path, preload=True, verbose=False)
C:\Users\Ania\AppData\Local\Temp\ipykernel_21216\702253907.py:23: RuntimeWarning: This filename (c:\Users\Ania\Desktop\studia\courses\6sem\thesis\RM\RM_1\RM_1st\RM10\RM10_102_epo.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif(file_path, preload=True, verbose=False)


KeyboardInterrupt: 

In [26]:
#Appointing iStims to condition and epoch variable
first_irr= ["101", "103","105","107"]
first_reg = ["102", "104","106","108"]
sec_sub = ["121","122","125","126"]
sec_obj = ["123","124","127","128"]
sec_irr = ["121","123","125","127"]
sec_reg = ["122","124","126","128"]

fmin = 2
fmax = 60
freqs = np.arange(fmin, fmax, 1.)
tmin = -1
tmax = 2.1

# specifying the output returns an averageTFR object (average across events)
rm_first_irr =  mne_multitaper_concate(first_irr, "RM_1/RM_1st/", freqs, fmin, fmax, tmin, tmax) # call to funciton above
rm_first_reg =  mne_multitaper_concate(first_reg, "RM_1/RM_1st", freqs, fmin, fmax, tmin, tmax) # call to funciton above
rm_sec_sub = mne_multitaper_concate(sec_sub, "RM_2/", freqs, fmin, fmax, tmin, tmax)
rm_sec_obj =  mne_multitaper_concate(sec_obj, "RM_2/", freqs, fmin, fmax, tmin, tmax) # call to funciton above
rm_sec_irr =  mne_multitaper_concate(sec_irr, "RM_2/", freqs, fmin, fmax, tmin, tmax) # call to funciton above
rm_sec_reg =  mne_multitaper_concate(sec_reg, "RM_2/", freqs, fmin, fmax, tmin, tmax) # call to funciton above



C:\Users\Ania\AppData\Local\Temp\ipykernel_13436\3315030010.py:48: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  concatenated_epochs = mne.concatenate_epochs(all_epochs)


Not setting metadata
2498 matching events found
Applying baseline correction (mode: mean)


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done  58 out of  58 | elapsed:  8.4min finished


Saved: RM_ambiguousunknownunknown-tfr.h5 and RM_ambiguousunknownunknown.npy


NameError: name 'tic' is not defined

In [ ]:
def mne_multitaper_part(iStims, path, freqs, fmin, fmax, tmin, tmax):

    '''
    This function creates the AveragerTFR Object for selected
    iStims per participant.
    '''

    n_cycles = freqs / 2.

    sample_rate = 500
    
    
    tic = time.perf_counter()


    
    # Adding iStims events per participant in one list
    for iPart in [1,2,6,8,9,10,11,12,13,14,16,18,21,23,28,30,32]:
        all_epochs = []
        file_list = []
        all_annotations = None
        dir_path = os.path.join(path, f"RM{str(iPart).zfill(2)}/")

        for iStim in iStims:
        
            # Check if the directory exists before listing files
            if os.path.exists(dir_path):
                # List files in the directory and filter by the stimulus and file extension
                file_list.extend([dir_path+f for f in os.listdir(dir_path) if iStim in f ])

        supersets = categorize_stimulus(iStim)
        if "XR" in supersets:
            XR.add(iStim)
        if "XA" in supersets:
            XA.add(iStim)
        if "XF" in supersets:
            XF.add(iStim)
    
        for file in file_list:
            file_epoch = mne.read_epochs(file, verbose=False)
            file_epoch.apply_baseline((-0.5, 0))

            if file_epoch.annotations is not None:
                if all_annotations is None:
                    all_annotations = file_epoch.annotations
                else:
                    all_annotations += file_epoch.annotations
                
        
            all_epochs.append(file_epoch)
        
        if all_epochs:
            # Concatenating epoch files
            concatenated_epochs = mne.concatenate_epochs(all_epochs)
       
            if all_annotations is not None:
                concatenated_epochs.set_annotations(all_annotations)
            
            # power analysis:
            power = concatenated_epochs.compute_tfr(method="multitaper", freqs=freqs, n_cycles=n_cycles, decim=10, return_itc=False, average=True, picks="eeg", n_jobs=-1)
            
    
            if path == "RM_1/":    
                if iStims == ["101", "103","105","107"]:
                    cond = "first_irr"
                elif iStims == ["102", "104","106","108"]:
                    cond = "first_reg"
            if path == "RM_2/":         
                if iStims == ["121","122","125","126"]:
                    cond = "sec_subj"
                elif iStims == ["123","124","127","128"]:
                    cond = "sec_obj"
                elif iStims == ["121","123","125","127"]:
                    cond = "sec_irr"
                elif iStims == ["122","124","126","128"]:
                    cond = "sec_reg"
                # Determining ambiguity category
            ambiguity = "non_ambiguous"  # Default value
            if any(categorize_stimulus(stim) for stim in iStims): #using the categorization function to check ambiguity
                ambiguity = "ambiguous"

        
        
            if not os.path.exists("Concatenated_events/without_excluded_part/RM"+str(iPart).zfill(2)):
                os.makedirs("Concatenated_events/without_excluded_part/RM"+str(iPart).zfill(2))
    
            tfr_name = "RM"+str(iPart).zfill(2)+"_"+cond+"-tfr.h5"
            os.chdir("Concatenated_events/without_excluded_part/RM"+str(iPart).zfill(2))
            mne.time_frequency.write_tfrs(tfr_name, power, overwrite=True, verbose=True)
            print("TFR file saved")    
            
            # extract power values to NumPy array
            final_array = power.crop(tmin = tmin, tmax = tmax, include_tmax = True).data # shape: chan x freq x time 
            #print(power.freqs) # debug step    
    
            # Saving the NumPy array file on the local output:
            file_name = "RM"+str(iPart).zfill(2)+'_'+cond+".npy"
            print("Saving file {}".format(file_name))
            np.save(file_name, final_array)
            print("NumPy array file saved")
            toc = time.perf_counter()
            print(f"Processed the file in {toc - tic:0.4f} seconds")
            os.chdir(dir)

    return power
            


In [ ]:

first_irr= ["101", "103","105","107"]
first_reg = ["102", "104","106","108"]
sec_sub = ["121","122","125","126"]
sec_obj = ["123","124","127","128"]
sec_irr = ["121","123","125","127"]
sec_reg = ["122","124","126","128"]

fmin = 2
fmax = 60
freqs = np.arange(fmin, fmax, 1.)
tmin = -0.75
tmax = 2

rm_first_irr =  mne_multitaper_part(first_irr, "RM_1/", freqs, fmin, fmax, tmin, tmax)
rm_first_reg =  mne_multitaper_part(first_reg, "RM_1/", freqs, fmin, fmax, tmin, tmax) 
rm_sec_sub =  mne_multitaper_part(sec_sub, "RM_2/", freqs, fmin, fmax, tmin, tmax) 
rm_sec_obj =  mne_multitaper_part(sec_obj, "RM_2/", freqs, fmin, fmax, tmin, tmax)
rm_sec_irr =  mne_multitaper_part(sec_irr, "RM_2/", freqs, fmin, fmax, tmin, tmax) 
rm_sec_reg =  mne_multitaper_part(sec_reg, "RM_2/", freqs, fmin, fmax, tmin, tmax)

UnboundLocalError: cannot access local variable 'power' where it is not associated with a value

In [ ]:
cond_1 = "sec_reg"
cond_2 = "sec_irr"
channels = ["Pz", "Fz"]
os.chdir("")
for iPart in range(1,33):

    if os.path.exists(f"Concatenated_events/RM{str(iPart).zfill(2)}"):
        rm_1 = mne.time_frequency.read_tfrs(f"Concatenated_events/RM{str(iPart).zfill(2)}/RM{str(iPart).zfill(2)}_{cond_1}-tfr.h5")
        rm_2 = mne.time_frequency.read_tfrs(f"Concatenated_events/RM{str(iPart).zfill(2)}/RM{str(iPart).zfill(2)}_{cond_2}-tfr.h5")

        rm_1.apply_baseline(baseline=(-0.5, 0), mode="logratio")
        rm_2.apply_baseline(baseline=(-0.5, 0), mode="logratio")
        
        tfr_list = [rm_1, rm_2]
        max_val = max([tfr.data.max() for tfr in tfr_list])
    
        
        for chan in channels:
    
            if not os.path.exists("Plots/Participants_per_Condition_Pz_Fz/RM"+str(iPart).zfill(2)):
                os.makedirs("Plots/Participants_per_Condition_Pz_Fz/RM"+str(iPart).zfill(2))
    
            os.chdir("Plots/Participants_per_Condition_Pz_Fz/RM"+str(iPart).zfill(2))
            
            
            rm_1.plot(tmax = 2.1, vlim=(-0.35,0.35), picks=[chan], title=f"RM{str(iPart).zfill(2)} - {cond_1} - {chan}")[0].savefig(f"RM{str(iPart).zfill(2)}_{cond_1}_{chan}.png", dpi=300)
            rm_2.plot(tmax = 2.1, vlim=(-0.35,0.35), picks=[chan], title=f"RM{str(iPart).zfill(2)} - {cond_2} - {chan}")[0].savefig(f"RM{str(iPart).zfill(2)}_{cond_2}_{chan}.png", dpi=300)
    
            os.chdir(dir)

    

# Average per Channel

In [ ]:
cond_1 = "first_reg"
cond_2 = "first_irr"


if os.path.exists(f"Concatenated_events/Grand_Average_Excluded"):
    rm_1 = mne.time_frequency.read_tfrs(f"Concatenated_events/Grand_Average_Excluded/RM_{cond_1}-tfr.h5")
    rm_2 = mne.time_frequency.read_tfrs(f"Concatenated_events/Grand_Average_Excluded/RM_{cond_2}-tfr.h5")
    
    for ch_index, ch_name in enumerate(ch_rm):
        
        # Create a single figure with 1 row and 2 columns
        fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(20, 5))
    
        # Plot the first condition in the first subplot
        rm_1.plot(picks=[ch_name], tmin=-0.75, tmax=2,vlim=(-0.35, 0.35), mode="logratio", baseline=(-0.5, 0), axes=axes[0], colorbar=True, show=False, verbose=False)
        axes[0].set_title(f"Regular - {ch_name}")
        
        # Plot the second condition in the second subplot
        rm_2.plot(picks=[ch_name], tmin=-0.75, tmax=2, vlim=(-0.35, 0.35), mode="logratio", baseline=(-0.5, 0), axes=axes[1], colorbar=True, show=False, verbose=False)
        axes[1].set_title(f"Irregular - {ch_name}")

        roi_params = [
            ((0, 2), 0.5, 6, 'red'),   # roi_1
            ((0.2, 9), 0.35, 7, 'black'),  # roi_2
            ((1, 9), 0.5, 7, 'green'),  # roi_3
            ((1, 9), 1, 7, 'blue'),    # roi_4
        ]
        
        for ax in fig.axes:
            for position, width, height, color in roi_params:
                roi = Rectangle(position, width, height, 
                                linewidth=1, edgecolor=color, facecolor='none', linestyle='--')
                ax.add_patch(roi)
        
        fig.suptitle(f"Comparison {ch_name}", fontsize=16)
        fig.savefig(f"Plots/Average_per_Channel_Comparison/First Epoch/{ch_name}.png")


    
    

# Average per Region

In [ ]:


cond_1 = "first_reg"
cond_2 = "first_irr"


if os.path.exists(f"Concatenated_events/Grand_Average_Excluded"):
    rm_1 = mne.time_frequency.read_tfrs(f"Concatenated_events/Grand_Average_Excluded/RM_{cond_1}-tfr.h5")
    rm_2 = mne.time_frequency.read_tfrs(f"Concatenated_events/Grand_Average_Excluded/RM_{cond_2}-tfr.h5")
    
    for subset, subset_name in zip([f_rm, c_rm, p_rm, la_rm, lp_rm, ra_rm, rp_rm], ["Frontal", "Central", "Parietal", "Left Anterior", "Left Posterior", "Right Anterior", "Right Posterior"]):
        
        # Create a single figure with 1 row and 2 columns
        fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(20, 5))
    
        # Plot the first condition in the first subplot
        rm_1.plot(picks=subset, combine="mean", tmin=-0.75, tmax=2,vlim=(-0.35, 0.35), mode="logratio", baseline=(-0.5, 0), axes=axes[0], colorbar=True, show=False, verbose=False)
        axes[0].set_title(f"Regular - {subset_name}")
        
        # Plot the second condition in the second subplot
        rm_2.plot(picks=subset, combine="mean", tmin=-0.75, tmax=2, vlim=(-0.35, 0.35), mode="logratio", baseline=(-0.5, 0), axes=axes[1], colorbar=True, show=False, verbose=False)
        axes[1].set_title(f"Irregular - {subset_name}")

        roi_params = [
            ((0, 2), 0.5, 6, 'red'),   # roi_1
            ((0.2, 9), 0.35, 7, 'black'),  # roi_2
            ((1, 9), 0.5, 7, 'green'),  # roi_3
            ((1, 9), 1, 7, 'blue'),    # roi_4
        ]
        
        for ax in fig.axes:
            for position, width, height, color in roi_params:
                roi = Rectangle(position, width, height, 
                                linewidth=1, edgecolor=color, facecolor='none', linestyle='--')
                ax.add_patch(roi)
        
        fig.suptitle(f"Comparison {subset_name}", fontsize=16)
        fig.savefig(f"Plots/Average_per_Region/{subset_name}.png")


    
    

# Channel per Participant 

In [ ]:
cond_1 = "first_reg"
cond_2 = "first_irr"

for iPart in range(1,33):
    if os.path.exists(f"Concatenated_events/without_excluded_part/RM{str(iPart).zfill(2)}"):
        rm_1 = mne.time_frequency.read_tfrs(f"Concatenated_events/without_excluded_part/RM{str(iPart).zfill(2)}/RM{str(iPart).zfill(2)}_{cond_1}-tfr.h5")
        rm_2 = mne.time_frequency.read_tfrs(f"Concatenated_events/without_excluded_part/RM{str(iPart).zfill(2)}/RM{str(iPart).zfill(2)}_{cond_2}-tfr.h5")
               
        for ch_index, ch_name in enumerate(ch_rm):

            save_path = os.path.join("Plots/Chan_per_part/First Epoch", f"RM{str(iPart).zfill(2)}")
            os.makedirs(save_path, exist_ok=True)
            
            # Create a single figure with 1 row and 2 columns
            fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(20, 5))
        
            # Plot the first condition in the first subplot
            rm_1.plot(picks=ch_name, tmin=-0.75, tmax=2,vlim=(-0.35, 0.35), mode="logratio", baseline=(-0.5, 0), axes=axes[0], colorbar=True, show=False, verbose=False)
            axes[0].set_title(f"Regular - {ch_name}")
            
            # Plot the second condition in the second subplot
            rm_2.plot(picks=ch_name, tmin=-0.75, tmax=2, vlim=(-0.35, 0.35), mode="logratio", baseline=(-0.5, 0), axes=axes[1], colorbar=True, show=False, verbose=False)
            axes[1].set_title(f"Irregular - {ch_name}")
    
            roi_params = [
                ((0, 2), 0.5, 6, 'red'),   # roi_1
                ((0.2, 9), 0.35, 6, 'black'),  # roi_2
                ((1, 9), 0.5, 6, 'green'),  # roi_3
                ((1, 9), 1, 6, 'blue'),    # roi_4
            ]
            
            for ax in fig.axes:
                for position, width, height, color in roi_params:
                    roi = Rectangle(position, width, height, 
                                    linewidth=1, edgecolor=color, facecolor='none', linestyle='--')
                    ax.add_patch(roi)
            
            fig.suptitle(f"Comparison for RM{str(iPart).zfill(2)} {ch_name}", fontsize=16)
            fig.savefig(f"Plots/Chan_per_part/First Epoch/RM{str(iPart).zfill(2)}/reg_irr_{ch_name}.png")




    
    

# Average of Subset of Channels per Particpant

In [ ]:
cond_1 = "first_reg"
cond_2 = "first_irr"

for iPart in range(1,33):
    if os.path.exists(f"Concatenated_events/without_excluded_part/RM{str(iPart).zfill(2)}"):
        rm_1 = mne.time_frequency.read_tfrs(f"Concatenated_events/without_excluded_part/RM{str(iPart).zfill(2)}/RM{str(iPart).zfill(2)}_{cond_1}-tfr.h5")
        rm_2 = mne.time_frequency.read_tfrs(f"Concatenated_events/without_excluded_part/RM{str(iPart).zfill(2)}/RM{str(iPart).zfill(2)}_{cond_2}-tfr.h5")

        for subset, subset_name in zip([f_rm, c_rm, p_rm, la_rm, lp_rm, ra_rm, rp_rm], ["Frontal", "Central", "Parietal", "Left Anterior", "Left Posterior", "Right Anterior", "Right Posterior"]):
       
            
            # Create a single figure with 1 row and 2 columns
            fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(20, 5))
        
            # Plot the first condition in the first subplot
            rm_1.plot(picks=subset, combine="mean", tmin=-0.75, tmax=2,vlim=(-0.35, 0.35), mode="logratio", baseline=(-0.5, 0), axes=axes[0], colorbar=True, show=False, verbose=False)
            axes[0].set_title(f"Regular - {subset_name}")
            
            # Plot the second condition in the second subplot
            rm_2.plot(picks=subset, combine="mean", tmin=-0.75, tmax=2, vlim=(-0.35, 0.35), mode="logratio", baseline=(-0.5, 0), axes=axes[1], colorbar=True, show=False, verbose=False)
            axes[1].set_title(f"Irregular - {subset_name}")
    
            roi_params = [
                ((0, 2), 0.5, 6, 'red'),   # roi_1
                ((0.2, 9), 0.35, 6, 'black'),  # roi_2
                ((1, 7), 0.5, 6, 'green'),  # roi_3
                ((1, 7), 1, 6, 'blue'),    # roi_4
            ]
            
            for ax in fig.axes:
                for position, width, height, color in roi_params:
                    roi = Rectangle(position, width, height, 
                                    linewidth=1, edgecolor=color, facecolor='none', linestyle='--')
                    ax.add_patch(roi)
            
            fig.suptitle(f"Comparison for RM{str(iPart).zfill(2)} {subset_name}", fontsize=16)
            fig.savefig(f"Plots/Average_Chan_Subset_per_Part/RM{str(iPart).zfill(2)}_reg_irr_{subset_name}.png")




    
    

# Difference per Region


In [ ]:

rm1 = mne.time_frequency.read_tfrs(f"Concatenated_events/Grand_Average_Excluded/RM_first_irr-tfr.h5")
rm2 = mne.time_frequency.read_tfrs(f"Concatenated_events/Grand_Average_Excluded/RM_first_reg-tfr.h5")

diff = rm2 -  rm1
for subset, subset_name in zip([f_rm, c_rm, p_rm, la_rm, lp_rm, ra_rm, rp_rm], ["Frontal", "Central", "Parietal", "Left Anterior", "Left Posterior", "Right Anterior", "Right Posterior"]):

    fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(10,5))

    diff.plot(axes=axes, picks=subset, title=f"Difference Regular - Irregular for {subset_name} Region", combine="mean", tmin=-0.75, tmax=2, colorbar=True, show=False,verbose=False)

    roi_params = [
        ((0, 2), 0.5, 6, 'red'),   # roi_1
        ((0.2, 9), 0.35, 7, 'black'),  # roi_2
        ((1, 9), 0.5, 7, 'green'),  # roi_3
        ((1, 9), 1, 7, 'blue'),    # roi_4
    ]
    
    for ax in fig.axes:
        for position, width, height, color in roi_params:
            roi = Rectangle(position, width, height, 
                            linewidth=1, edgecolor=color, facecolor='none', linestyle='--')
            ax.add_patch(roi)

# Paired T-Test

In [ ]:

def analyze_tfr(channels, subset_name, roi_definitions, participants_range=range(1, 33)):
    results = {}
    cond_reg = "first_reg"
    cond_irr = "first_irr"

    for iPart in participants_range:
        participant_name = f"RM{str(iPart).zfill(2)}"
        participant_path = f"Concatenated_events/without_excluded_part/{participant_name}"
        
        
        if os.path.exists(participant_path):
            # Read the TFR data for both regular and irregular conditions
            
            rm_reg = mne.time_frequency.read_tfrs(f"{participant_path}/{participant_name}_{cond_reg}-tfr.h5", verbose=False)
            rm_irr = mne.time_frequency.read_tfrs(f"{participant_path}/{participant_name}_{cond_irr}-tfr.h5", verbose=False)

            # Select channels within the subset and average over the selected channels
            
            tfr_reg_subset = rm_reg.pick(picks=channels).data.mean(axis=0)
            tfr_irr_subset = rm_irr.pick(picks=channels).data.mean(axis=0) 
    
            for roi in roi_definitions:
                
                # Define masks to select specific time and frequency ranges for the regular condition
                baseline_mask_reg = (rm_reg.times >= roi["baseline_time"][0]) & (rm_reg.times <= roi["baseline_time"][1])
                time_mask_reg = (rm_reg.times >= roi["time_range"][0]) & (rm_reg.times <= roi["time_range"][1])
                freq_mask_reg = (rm_reg.freqs >= roi["freq_range"][0]) & (rm_reg.freqs <= roi["freq_range"][1])
    
                # Define masks to select specific time and frequency ranges for the irregular condition
                baseline_mask_irr = (rm_irr.times >= roi["baseline_time"][0]) & (rm_irr.times <= roi["baseline_time"][1])
                time_mask_irr = (rm_irr.times >= roi["time_range"][0]) & (rm_irr.times <= roi["time_range"][1])
                freq_mask_irr = (rm_irr.freqs >= roi["freq_range"][0]) & (rm_irr.freqs <= roi["freq_range"][1])
    
                # Calculate the average power within the time-frequency zone for each condition                
                avg_tfr_baseline_reg = tfr_reg_subset[freq_mask_reg][:, baseline_mask_reg].mean() 
                avg_tfr_reg = tfr_reg_subset[freq_mask_reg][:, time_mask_reg].mean() 
    
                avg_tfr_baseline_irr = tfr_irr_subset[freq_mask_irr][:, baseline_mask_irr].mean()  
                avg_tfr_irr = tfr_irr_subset[freq_mask_irr][:, time_mask_irr].mean()  
    
                # Store results for paired t-test
                if (subset_name, roi["name"]) not in results:
                    results[(subset_name, roi["name"])] = {"baseline_reg": [], "regular": [], "baseline_irr": [], "irregular": []}
                
                # Append the calculated averages to the results dictionary
                results[(subset_name, roi["name"])]['baseline_reg'].append(avg_tfr_baseline_reg)                
                results[(subset_name, roi["name"])]['regular'].append(avg_tfr_reg)
                results[(subset_name, roi["name"])]['baseline_irr'].append(avg_tfr_baseline_irr)
                results[(subset_name, roi["name"])]['irregular'].append(avg_tfr_irr)

    # Perform paired t-tests for each subset and ROI
    for key, value in results.items():
        subset_name, roi_name = key

        
        # Paired t-test between baseline and experimental condition for regular
        t_stat_reg, p_value_reg = stats.ttest_rel(value["baseline_reg"], value["regular"])
        
        # Paired t-test between baseline and experimental condition for irregular
        t_stat_irr, p_value_irr = stats.ttest_rel(value["baseline_irr"], value["irregular"])
        
        # Paired t-test between regular and irregular conditions (regardless of baseline)
        t_stat_cond, p_value_cond = stats.ttest_rel(value["regular"], value["irregular"])

        # Print the results of the t-tests
        print(f"{subset_name} - {roi_name}:")
        print(f"  Baseline vs Regular: T-statistic: {t_stat_reg}, p-value: {p_value_reg}")
        print(f"  Baseline vs Irregular: T-statistic: {t_stat_irr}, p-value: {p_value_irr}")
        print(f"  Regular vs Irregular: T-statistic: {t_stat_cond}, p-value: {p_value_cond}")
        print()



In [ ]:
cond_reg = "first_reg"
cond_irr = "first_irr"

#Define ROIs
roi_definitions = [
    {"baseline_time": (-0.3, -0.2), "time_range": (0, 0.5), "freq_range": (2, 8), "name": "ROI 1"},
    {"baseline_time": (-0.3, -0.2), "time_range": (0.2, 0.55), "freq_range": (9, 16), "name": "ROI 2"},
    {"baseline_time": (-0.3, -0.2), "time_range": (1.0, 1.5), "freq_range": (7, 13), "name": "ROI 3"},
    {"baseline_time": (-0.3, -0.2), "time_range": (1.0, 2.0), "freq_range": (7, 13), "name": "ROI 4"}
]

analyze_tfr(f_rm, "Frontal", roi_definitions)
analyze_tfr(p_rm, "Parietal", roi_definitions)
analyze_tfr(c_rm, "Central", roi_definitions)
analyze_tfr(la_rm, "Left Anterior", roi_definitions)
analyze_tfr(ra_rm, "Right Anterior", roi_definitions)
analyze_tfr(lp_rm, "Left Posterior", roi_definitions)
analyze_tfr(rp_rm, "Right Posterior", roi_definitions)

Frontal - ROI 1:
  Baseline vs Regular: T-statistic: -2.970445716420161, p-value: 0.009018332467875012
  Baseline vs Irregular: T-statistic: -1.2291085690873798, p-value: 0.23680160434032532
  Regular vs Irregular: T-statistic: -1.4605288183318017, p-value: 0.1635030187792237

Frontal - ROI 2:
  Baseline vs Regular: T-statistic: 1.1536574795776202, p-value: 0.26558342723186457
  Baseline vs Irregular: T-statistic: 1.0748518324776855, p-value: 0.2983835208656376
  Regular vs Irregular: T-statistic: 1.1220322390687454, p-value: 0.27840558058376547

Frontal - ROI 3:
  Baseline vs Regular: T-statistic: -1.0886280329653917, p-value: 0.2924443375062148
  Baseline vs Irregular: T-statistic: -1.099012071366742, p-value: 0.28802540663915643
  Regular vs Irregular: T-statistic: 2.346125936123713, p-value: 0.03218138702214179

Frontal - ROI 4:
  Baseline vs Regular: T-statistic: -1.0968246830075616, p-value: 0.2889521265519259
  Baseline vs Irregular: T-statistic: -1.0928899490724953, p-value: 0.

# Mixed-effects model

In [ ]:
import os
import numpy as np
import pandas as pd
import mne
import statsmodels.formula.api as smf

def prepare_data(channels, subset_name, roi_definitions, participants_range=range(1, 33)):
    """Prepare the data for mixed-effects modeling."""
    data = []
    cond_reg = "first_reg"
    cond_irr = "first_irr"

    for iPart in participants_range:
        participant_name = f"RM{str(iPart).zfill(2)}"
        participant_path = f"Concatenated_events/without_excluded_part/{participant_name}"
        
        if os.path.exists(participant_path):
            # Read the TFR data for both regular and irregular conditions
            rm_reg = mne.time_frequency.read_tfrs(f"{participant_path}/{participant_name}_{cond_reg}-tfr.h5", verbose=False)
            rm_irr = mne.time_frequency.read_tfrs(f"{participant_path}/{participant_name}_{cond_irr}-tfr.h5", verbose=False)

            # Select channels within the subset and average over the selected channels
            tfr_reg_subset = rm_reg.pick(picks=channels).data.mean(axis=0)
            tfr_irr_subset = rm_irr.pick(picks=channels).data.mean(axis=0)
    
            for roi in roi_definitions:
                # Define masks for time and frequency ranges
                baseline_mask_reg = (rm_reg.times >= roi["baseline_time"][0]) & (rm_reg.times <= roi["baseline_time"][1])
                time_mask_reg = (rm_reg.times >= roi["time_range"][0]) & (rm_reg.times <= roi["time_range"][1])
                freq_mask_reg = (rm_reg.freqs >= roi["freq_range"][0]) & (rm_reg.freqs <= roi["freq_range"][1])

                baseline_mask_irr = (rm_irr.times >= roi["baseline_time"][0]) & (rm_irr.times <= roi["baseline_time"][1])
                time_mask_irr = (rm_irr.times >= roi["time_range"][0]) & (rm_irr.times <= roi["time_range"][1])
                freq_mask_irr = (rm_irr.freqs >= roi["freq_range"][0]) & (rm_irr.freqs <= roi["freq_range"][1])

                # Compute averages
                avg_baseline_reg = tfr_reg_subset[freq_mask_reg][:, baseline_mask_reg].mean()
                avg_reg = tfr_reg_subset[freq_mask_reg][:, time_mask_reg].mean()
                avg_baseline_irr = tfr_irr_subset[freq_mask_irr][:, baseline_mask_irr].mean()
                avg_irr = tfr_irr_subset[freq_mask_irr][:, time_mask_irr].mean()

                # Append data for each condition
                data.extend([
                    {"Participant": participant_name, "Condition": "Baseline", "Region": subset_name, "Power": avg_baseline_reg, "ROI": roi["name"], "Hemisphere": "Both"},
                    {"Participant": participant_name, "Condition": "Regular", "Region": subset_name, "Power": avg_reg, "ROI": roi["name"], "Hemisphere": "Both"},
                    {"Participant": participant_name, "Condition": "Baseline", "Region": subset_name, "Power": avg_baseline_irr, "ROI": roi["name"], "Hemisphere": "Both"},
                    {"Participant": participant_name, "Condition": "Irregular", "Region": subset_name, "Power": avg_irr, "ROI": roi["name"], "Hemisphere": "Both"},
                ])
    
    return pd.DataFrame(data)


# Define ROIs
roi_definitions = [
    {"baseline_time": (-0.3, -0.2), "time_range": (0, 0.5), "freq_range": (2, 8), "name": "ROI 1"},
    {"baseline_time": (-0.3, -0.2), "time_range": (0.2, 0.55), "freq_range": (9, 16), "name": "ROI 2"},
    {"baseline_time": (-0.3, -0.2), "time_range": (1.0, 1.5), "freq_range": (7, 13), "name": "ROI 3"},
    {"baseline_time": (-0.3, -0.2), "time_range": (1.0, 2.0), "freq_range": (7, 13), "name": "ROI 4"}
]

# Prepare the data
data_frontal = prepare_data(f_rm, "Frontal", roi_definitions)
data_parietal = prepare_data(p_rm, "Parietal", roi_definitions)
data_central = prepare_data(c_rm, "Central", roi_definitions)

# Combine all data
all_data = pd.concat([data_frontal, data_parietal, data_central])

In [ ]:
import pandas as pd
import statsmodels.formula.api as smf
from statsmodels.stats.multitest import multipletests

def pairwise_comparisons(data):
    """
    Perform pairwise comparisons for Baseline vs Regular, Baseline vs Irregular, 
    and Regular vs Irregular within each ROI and region, with FDR correction.
    """
    results = []
    
    for roi in data["ROI"].unique():
        for region in data["Region"].unique():
            # Subset data for the current ROI and region
            subset_data = data[(data["ROI"] == roi) & (data["Region"] == region)]
            
            print(f"\nPerforming pairwise comparisons for ROI {roi} in Region {region}...")
            
            # Pairwise comparisons
            comparisons = [
                ("Baseline", "Regular"),
                ("Baseline", "Irregular"),
                ("Regular", "Irregular")
            ]
            
            for condition1, condition2 in comparisons:
                # Subset data for the two conditions
                pair_data = subset_data[subset_data["Condition"].isin([condition1, condition2])]
                
                if len(pair_data) < 3:
                    print(f"Skipping comparison {condition1} vs {condition2} for ROI {roi}, Region {region} due to insufficient data.")
                    continue
                
                try:
                    # Mixed-effects model for pairwise comparison
                    model = smf.mixedlm("Power ~ Condition", pair_data, groups=pair_data["Participant"])
                    result = model.fit(reml=True, method='lbfgs')
                    
                    # Extract parameter names dynamically
                    param_name = f"Condition[T.{condition2}]"
                    coef = result.params.get(param_name, np.nan)
                    pval = result.pvalues.get(param_name, np.nan)
                    
                    # Store results
                    results.append({
                        "ROI": roi,
                        "Region": region,
                        "Comparison": f"{condition1} vs {condition2}",
                        "Coef": coef,
                        "P-Value": pval
                    })
                    
                    # Print summary
                    print(f"Comparison {condition1} vs {condition2}:")
                    print(result.summary())
                except Exception as e:
                    print(f"Error fitting model for comparison {condition1} vs {condition2} in ROI {roi}, Region {region}: {e}")
    
    # Convert results to a DataFrame for easier interpretation
    results_df = pd.DataFrame(results)
    
    # Apply FDR correction to P-Values
    if not results_df.empty:
        valid_pvals_idx = results_df["P-Value"].notna()
        p_values = results_df.loc[valid_pvals_idx, "P-Value"].values
        _, corrected_p_values, _, _ = multipletests(p_values, method="fdr_bh")
        
        # Update only rows with valid p-values
        results_df.loc[valid_pvals_idx, "Corrected P-Value"] = corrected_p_values
    else:
        results_df["Corrected P-Value"] = None
    
    return results_df


# Example: Assuming 'all_data' is your combined DataFrame
results_df = pairwise_comparisons(all_data)

# Save or display results
print(results_df)
results_df.to_csv("pairwise_comparisons_results_corrected.csv", index=False)



Performing pairwise comparisons for ROI ROI 1 in Region Frontal...
Comparison Baseline vs Regular:
            Mixed Linear Model Regression Results
Model:               MixedLM   Dependent Variable:   Power   
No. Observations:    51        Method:               REML    
No. Groups:          17        Scale:                0.0000  
Min. group size:     3         Log-Likelihood:       910.7462
Max. group size:     3         Converged:            Yes     
Mean group size:     3.0                                     
-------------------------------------------------------------
                     Coef. Std.Err.   z   P>|z| [0.025 0.975]
-------------------------------------------------------------
Intercept            0.000    0.000 2.231 0.026  0.000  0.000
Condition[T.Regular] 0.000    0.000 1.210 0.226 -0.000  0.000
Group Var            0.000    0.000                          

Comparison Baseline vs Irregular:
             Mixed Linear Model Regression Results
Model:              

/home/pdrabbe/.local/lib/python3.10/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
/home/pdrabbe/.local/lib/python3.10/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
/home/pdrabbe/.local/lib/python3.10/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
/home/pdrabbe/.local/lib/python3.10/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


Comparison Baseline vs Regular:
            Mixed Linear Model Regression Results
Model:               MixedLM   Dependent Variable:   Power   
No. Observations:    51        Method:               REML    
No. Groups:          17        Scale:                0.0000  
Min. group size:     3         Log-Likelihood:       914.4835
Max. group size:     3         Converged:            Yes     
Mean group size:     3.0                                     
-------------------------------------------------------------
                     Coef. Std.Err.   z   P>|z| [0.025 0.975]
-------------------------------------------------------------
Intercept            0.000    0.000 1.842 0.065 -0.000  0.000
Condition[T.Regular] 0.000    0.000 2.395 0.017  0.000  0.000
Group Var            0.000    0.000                          

Comparison Baseline vs Irregular:
             Mixed Linear Model Regression Results
Model:               MixedLM    Dependent Variable:    Power   
No. Observations:    51 

/home/pdrabbe/.local/lib/python3.10/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
/home/pdrabbe/.local/lib/python3.10/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
/home/pdrabbe/.local/lib/python3.10/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
/home/pdrabbe/.local/lib/python3.10/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


Comparison Baseline vs Irregular:
             Mixed Linear Model Regression Results
Model:               MixedLM    Dependent Variable:    Power   
No. Observations:    51         Method:                REML    
No. Groups:          17         Scale:                 0.0000  
Min. group size:     3          Log-Likelihood:        915.7181
Max. group size:     3          Converged:             Yes     
Mean group size:     3.0                                       
---------------------------------------------------------------
                       Coef. Std.Err.   z   P>|z| [0.025 0.975]
---------------------------------------------------------------
Intercept              0.000    0.000 2.008 0.045  0.000  0.000
Condition[T.Irregular] 0.000    0.000 3.545 0.000  0.000  0.000
Group Var              0.000    0.000                          

Comparison Regular vs Irregular:
             Mixed Linear Model Regression Results
Model:               MixedLM    Dependent Variable:    Power  

...

Comparison Baseline vs Regular:
            Mixed Linear Model Regression Results
Model:               MixedLM   Dependent Variable:   Power   
No. Observations:    51        Method:               REML    
No. Groups:          17        Scale:                0.0000  
Min. group size:     3         Log-Likelihood:       886.1562
Max. group size:     3         Converged:            Yes     
Mean group size:     3.0                                     
-------------------------------------------------------------
                     Coef. Std.Err.   z   P>|z| [0.025 0.975]
-------------------------------------------------------------
Intercept            0.000    0.000 1.456 0.145 -0.000  0.000
Condition[T.Regular] 0.000    0.000 1.618 0.106 -0.000  0.000
Group Var            0.000    0.000                          

Comparison Baseline vs Irregular:
             Mixed Linear Model Regression Results
Model:               MixedLM    Dependent Variable:    Power   
No. Observations:    51 

/home/pdrabbe/.local/lib/python3.10/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
/home/pdrabbe/.local/lib/python3.10/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
/home/pdrabbe/.local/lib/python3.10/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
/home/pdrabbe/.local/lib/python3.10/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


Comparison Baseline vs Irregular:
             Mixed Linear Model Regression Results
Model:               MixedLM    Dependent Variable:    Power   
No. Observations:    51         Method:                REML    
No. Groups:          17         Scale:                 0.0000  
Min. group size:     3          Log-Likelihood:        872.7078
Max. group size:     3          Converged:             Yes     
Mean group size:     3.0                                       
---------------------------------------------------------------
                       Coef. Std.Err.   z   P>|z| [0.025 0.975]
---------------------------------------------------------------
Intercept              0.000    0.000 1.772 0.076 -0.000  0.000
Condition[T.Irregular] 0.000    0.000 1.299 0.194 -0.000  0.000
Group Var              0.000    0.000                          

Comparison Regular vs Irregular:
            Mixed Linear Model Regression Results
Model:               MixedLM   Dependent Variable:   Power   
N

/home/pdrabbe/.local/lib/python3.10/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
/home/pdrabbe/.local/lib/python3.10/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
/home/pdrabbe/.local/lib/python3.10/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
/home/pdrabbe/.local/lib/python3.10/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


Comparison Regular vs Irregular:
            Mixed Linear Model Regression Results
Model:               MixedLM   Dependent Variable:   Power   
No. Observations:    34        Method:               REML    
No. Groups:          17        Scale:                0.0000  
Min. group size:     2         Log-Likelihood:       593.5107
Max. group size:     2         Converged:            Yes     
Mean group size:     2.0                                     
-------------------------------------------------------------
                     Coef. Std.Err.   z   P>|z| [0.025 0.975]
-------------------------------------------------------------
Intercept            0.000    0.000 1.562 0.118 -0.000  0.000
Condition[T.Regular] 0.000    0.000 2.213 0.027  0.000  0.000
Group Var            0.000    0.000                          

      ROI    Region             Comparison          Coef       P-Value  \
0   ROI 1   Frontal    Baseline vs Regular  2.833935e-10  2.264299e-01   
1   ROI 1   Frontal  Bas